In [1]:
import os
import h5py
import numpy as np
from skimage.transform import resize

In [2]:
data_dir = 'C:\\Users\\VANSH GARG\\.cache\\kagglehub\\datasets\\awsaf49\\brats2020-training-data\\versions\\3\\BraTS2020_training_data\\content\\data'
h5_files = [f for f in sorted(os.listdir(data_dir)) if f.endswith('.h5')]

In [ ]:
def data_generation(files):
    data_sorted=[0]*(155*369)
    for i in files:
        L=i.split("_")
        k=int(L[-1].split(".")[0])
        n=int(L[1])
        data_sorted[(155*(n-1))+k]=i
    img_slices=[]
    msk_slices=[]
    j=0
    with h5py.File("brain_tumor_data4.h5", "w") as hf:
        hf.create_dataset("X", shape=(369, 128, 128, 32, 4), dtype='float16', chunks=True)
        hf.create_dataset("Y", shape=(369, 128, 128, 32), dtype='float16', chunks=True)
        for k,i in enumerate(data_sorted):
            j+=1
            file=os.path.join(data_dir,i)
            with h5py.File(file, 'r') as f:

                image_data=np.array(f['image'],dtype='float16')
                #image_data= resize(image_data,(*(64,64),image_data.shape[2]),preserve_range=True,anti_aliasing=True)
                image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data) + 1e-6)
                mask_data=np.array(f['mask'],dtype='float16')
                mask_data = np.mean(mask_data, axis=-1)
                #mask_data=resize(mask_data,(32,32),preserve_range=True,anti_aliasing=False)
                mask_data = (mask_data - np.min(mask_data)) / (np.max(mask_data) - np.min(mask_data) + 1e-6)
                if np.max(mask_data) > 0:
                    img_slices.append(image_data)
                    msk_slices.append(mask_data)
                if j==155:
                    img_voxel=np.stack(img_slices,axis=2)
                    msk_voxel=np.stack(msk_slices,axis=2)
                    img_voxel= resize(img_voxel,(128,128,32,4),preserve_range=True,anti_aliasing=True)
                    msk_voxel= resize(msk_voxel,(128,128,32),preserve_range=True,anti_aliasing=True)
                    hf["X"][(k//155)] = img_voxel
                    hf["Y"][(k//155)] = msk_voxel
                    img_slices=[]
                    msk_slices=[]
                    j=0
                    continue    
data_generation(h5_files)
